In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# import mne
# import numpy as np
# from copy import deepcopy
# import matplotlib.pyplot as plt
import sys; sys.path.insert(0, '../')
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet.forward import create_forward_model, get_info
from scipy.stats import pearsonr
from matplotlib import pyplot as plt
plot_params = dict(surface='white', hemi='both', verbose=0)

## Create Forward model
First we create a template forward model which comes with the esinet package

In [2]:
info = get_info(sfreq=100)
fwd = create_forward_model(sampling="ico3", info=info)
leadfield = fwd["sol"]["data"]

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    1.6s remaining:    2.7s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    1.6s remaining:    0.9s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    1.7s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.0s finished


# Models

## The Generator

In [256]:
import mne
from scipy.sparse.csgraph import laplacian

adjacency = mne.spatial_src_adjacency(fwd['src']).toarray()
laplace_operator = laplacian(adjacency)
laplace_operator.shape

-- number of adjacent vertices : 1284


(1284, 1284)

In [286]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, InputLayer, Input, Lambda, LayerNormalization
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
import mne
from scipy.sparse.csgraph import laplacian

adjacency = mne.spatial_src_adjacency(fwd['src']).toarray()
laplace_operator = laplacian(adjacency)

def custom_gan_loss(y_true, y_hat):
    # error = -tf.keras.losses.CosineSimilarity()(y_hat[0], y_hat[1])
    error = -tf.keras.losses.mean_squared_error(y_hat[0], y_hat[1])
    # blur = tf.math.count_nonzero(y_hat[0], dtype=tf.float32) / tf.cast(tf.size(y_hat[0]), dtype=tf.float32)

    return error # + blur

def define_models(latent_dim, hidden_units=200):
    n_chans, n_dipoles = leadfield.shape
    leadfield_ = tf.cast(leadfield, dtype=tf.float32)

    inputs = tf.keras.Input(shape=(latent_dim), name='Input_Generator')

    # G MODEL
    fc = Dense(latent_dim, name="HL_G")(inputs)
    gen_out = Dense(n_dipoles, name="Output_Generator", activation="tanh")(fc)
    # gen_out = Lambda(lambda x: tf.cast(x>K.max(K.abs(x))*0.8, dtype=x.dtype) * x, output_shape=(None, n_dipoles))(gen_out)
    g_model = tf.keras.Model(inputs=inputs, outputs=gen_out, name='Generator')
    # g_model.build(input_shape=(latent_dim))
    
    # D MODEL
    input_shape = (None, n_chans)
    inputs2 = tf.keras.Input(shape=input_shape, name='Input_Discriminator')
    fc2 = Dense(hidden_units, name="HL_D")(inputs2)
    out = Dense(n_dipoles, name="Output_Final")(fc2)
    d_model = tf.keras.Model(inputs=inputs2, outputs=out, name='Discriminator')
    # d_model.build(input_shape=(latent_dim))
    
    lam = Lambda(lambda x: tf.transpose(tf.linalg.matmul(leadfield_, tf.transpose(x))), output_shape=(None, n_chans))
    inputs = tf.keras.Input(shape=(latent_dim), name='Input_Generator')
    # d_model.trainable = False
    output_1 = g_model(inputs)
    eeg = lam(output_1)
    eeg_normed = LayerNormalization()(eeg)
    output_3 = d_model(eeg_normed)
    gan_model = Model(inputs, [output_1, output_3])

    g_model.compile(loss=custom_gan_loss, optimizer="adam")
    # d_model.compile(loss=tf.keras.losses.CosineSimilarity(), optimizer="adam")
    d_model.compile(loss="mse", optimizer="adam")
    
    gan_model.compile(loss=custom_gan_loss, optimizer="adam")
    
    return g_model, d_model, gan_model

def prep_data(X, y):
    X = np.stack([(x - np.mean(x)) / np.std(x) for x in X], axis=0)
    y = np.stack([(x / np.max(abs(x))) for x in y], axis=0)

    if len(X.shape) == 2:
        X = np.expand_dims(X, axis=-1)
        y = np.expand_dims(y, axis=-1)
    X = np.swapaxes(X, 1,2)
    y = np.swapaxes(y, 1,2)
    return X, y

def generate_samples(g_model, batch_size, latent_dim):
    x_input = np.random.randn(batch_size, latent_dim)
    sources = g_model.predict(x_input)
    return sources


n_epochs = 200
batch_size = 32
batch_number = 10
latent_dim = 100
g_model, d_model, gan_model = define_models(latent_dim, hidden_units=latent_dim)

-- number of adjacent vertices : 1284


## old functions

In [139]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, InputLayer, Input, Lambda
import numpy as np

n_chans, n_dipoles = leadfield.shape

def define_generator(latent_dim):
    g_model = tf.keras.Sequential()
    input_shape = (None, latent_dim)
    g_model.add(InputLayer(input_shape=input_shape))
    g_model.add(Dense(latent_dim, name="HL1"))
    g_model.add(Dense(n_dipoles, name="Output"))
    # g_model.build()
    # g_model.compile(optimizer='adam', loss="mse")
    # g_model.summary()
    return g_model
    
def define_discriminator(hidden_units=100):
    input_shape = (None, n_chans)
    d_model = tf.keras.Sequential()
    d_model.add(InputLayer(input_shape=input_shape))
    d_model.add(Dense(hidden_units, name="HL1"))
    d_model.add(Dense(n_dipoles, name="Output"))
    d_model.build()
    d_model.compile(optimizer='adam', loss=tf.keras.losses.CosineSimilarity())
    # d_model.summary()
    return d_model

def define_gan(g_model, d_model, latent_dim):
    leadfield_ = tf.cast(leadfield, dtype=tf.float32)
    d_model.trainable = False
    
    input_shape = (None, latent_dim)
    
    lam = Lambda(lambda x: tf.transpose(tf.linalg.matmul(leadfield_, tf.transpose(x))), output_shape=(None, n_chans))(g_model.output)
    print(lam)
    discriminator = d_model(lam)
    model = tf.keras.Model(inputs=g_model.input, outputs=[d_model.output, g_model.output], name='Contextualizer')


    # model = tf.keras.Sequential()
    # model.add(g_model)
    # model.add(Lambda(lambda x: tf.linalg.matmul(leadfield_, x)))
    # model.add(d_model)
    # model.compile(loss='binary_crossentropy', optimizer="adam")

    return model
 



KerasTensor(type_spec=TensorSpec(shape=(None, 61, 1284), dtype=tf.float32, name=None), name='lambda_89/transpose_1:0', description="created by layer 'lambda_89'")


ValueError: Input 0 of layer HL1 is incompatible with the layer: expected axis -1 of input shape to have value 61 but received input with shape (None, 61, 1284)

## Train

In [289]:
n_epochs = 10000
batch_size = 32
latent_dim = 1000
g_model, d_model, gan_model = define_models(latent_dim, hidden_units=latent_dim)

for i in range(n_epochs):
    y = generate_samples(g_model, batch_size, latent_dim)
    X = (leadfield @ y.T).T
    X, y = prep_data(X,y)
    d_model.trainable = True
    d_loss = d_model.train_on_batch(X, y)
    if i%100 == 0:    
        x_input = np.random.randn(batch_size, latent_dim)
        X = np.zeros((batch_size, n_dipoles))
        d_model.trainable = False
        gan_loss = gan_model.train_on_batch(x_input, X)[0]
        print(f'disc-loss: {d_loss:.2f}, gan-loss: {gan_loss:.2f}')
    else:
        print(f'disc-loss: {d_loss:.2f}')




disc-loss: 0.47, gan-loss: -0.83
disc-loss: 0.50
disc-loss: 0.42
disc-loss: 0.40
disc-loss: 0.39
disc-loss: 0.40
disc-loss: 0.40
disc-loss: 0.40
disc-loss: 0.40
disc-loss: 0.39
disc-loss: 0.39
disc-loss: 0.39
disc-loss: 0.38
disc-loss: 0.39
disc-loss: 0.38
disc-loss: 0.38
disc-loss: 0.38
disc-loss: 0.38
disc-loss: 0.38
disc-loss: 0.37
disc-loss: 0.37
disc-loss: 0.38
disc-loss: 0.37
disc-loss: 0.37
disc-loss: 0.37
disc-loss: 0.37
disc-loss: 0.37
disc-loss: 0.37
disc-loss: 0.37
disc-loss: 0.37
disc-loss: 0.37
disc-loss: 0.37
disc-loss: 0.37
disc-loss: 0.37
disc-loss: 0.37
disc-loss: 0.36
disc-loss: 0.37
disc-loss: 0.37
disc-loss: 0.37
disc-loss: 0.36
disc-loss: 0.36
disc-loss: 0.36
disc-loss: 0.36
disc-loss: 0.37
disc-loss: 0.36
disc-loss: 0.36
disc-loss: 0.36
disc-loss: 0.36
disc-loss: 0.36
disc-loss: 0.36
disc-loss: 0.36
disc-loss: 0.37
disc-loss: 0.37
disc-loss: 0.36
disc-loss: 0.36
disc-loss: 0.36
disc-loss: 0.37
disc-loss: 0.36
disc-loss: 0.36
disc-loss: 0.36
disc-loss: 0.36
disc-lo

KeyboardInterrupt: 

# Evaluate

In [290]:
n_samples = 2
# settings = dict(duration_of_trial=0.25, extents=(1,40), number_of_sources=(1,15), target_snr=(2, 15))
settings = dict(duration_of_trial=0.1, extents=25, number_of_sources=1, target_snr=1e99)

sim_test = Simulation(fwd, info, settings=settings).simulate(n_samples=n_samples)
def prep_data_sim(sim):
    X = np.squeeze(np.stack([eeg.average().data for eeg in sim.eeg_data]))
    X = np.stack([(x - np.mean(x)) / np.std(x) for x in X], axis=0)
    y = np.squeeze(np.stack([src.data for src in sim.source_data]))
    y = np.stack([(x / np.max(abs(x))) for x in y], axis=0)

    if len(X.shape) == 2:
        X = np.expand_dims(X, axis=-1)
        y = np.expand_dims(y, axis=-1)
    X = np.swapaxes(X, 1,2)
    y = np.swapaxes(y, 1,2)
    return X, y
X, y = prep_data_sim(sim_test)

y_hat = d_model.predict(X)
y_hat.shape
stc = sim_test.source_data[0]
stc.plot(**plot_params)

stc_hat = stc.copy()
stc_hat.data = y_hat[0].T
stc_hat.plot(**plot_params)
from scipy.stats import pearsonr
r,p = pearsonr(stc.data.flatten(), stc_hat.data.flatten())
print(f'r={r:.2f}, p={p:.4f}')

Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 181.92it/s]


r=0.16, p=0.0000


# FC

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, TimeDistributed, Bidirectional, LSTM, GRU, multiply, Activation
from tensorflow.keras.regularizers import l1
from esinet.losses import nmae_loss
leadfield, pos = util.unpack_fwd(fwd)[1:3]
n_channels, n_dipoles = leadfield.shape
input_shape = (None, None, n_channels)
tf.keras.backend.set_image_data_format('channels_last')

n_dense_units = 600
n_lstm_units = 30
activation_function = "tanh"
batch_size = 32
epochs = 30
dropout = 0.1

inputs = tf.keras.Input(shape=(None, n_channels), name='Input')
fc1 = TimeDistributed(Dense(n_dense_units, 
            activation=activation_function), 
            name='FC1')(inputs)
direct_out = TimeDistributed(Dense(n_dipoles, 
            activation="linear"),
            name='FC2')(fc1)


model3 = tf.keras.Model(inputs=inputs, outputs=direct_out, name='FC')


model3.compile(loss=tf.keras.losses.CosineSimilarity(), optimizer="adam")

model3.summary()
model3.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.15)

Model: "FC"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, None, 61)]        0         
                                                                 
 FC1 (TimeDistributed)       (None, None, 600)         37200     
                                                                 
 FC2 (TimeDistributed)       (None, None, 1284)        771684    
                                                                 
Total params: 808,884
Trainable params: 808,884
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
399/399 [==============================] - 3s 7ms/step - loss: -0.2897 - val_loss: -0.3501
Epoch 2/30
399/399 [==============================] - 2s 5ms/step - loss: -0.3597 - val_loss: -0.3755
Epoch 3/30
399/399 [==============================] - 2s 5ms/step - loss: -0.3755 - val_loss: -0.3851
Epoch 4/30
399/3